In [2]:
import timeit

from numba import njit
import torch
import numpy as np

# 1. Vectorization


### 1.1 Sum of array elements


In [2]:
@njit
def sum_numba(x):
    total = 0.0
    for i in range(x.shape[0]):
        total += x[i]
    return total


def sum_numpy(x):
    return np.sum(x)


@njit
def sum_numpy_numba(x):
    return np.sum(x)


def sum_torch(x):
    x_torch = torch.from_numpy(x)
    return torch.sum(x_torch).item()


def sum_iterative(x):
    total = 0.0
    for value in x:
        total += value
    return total


def sum_builtin(x):
    return sum(x)


sum_functions = {
    "Numba Iterative": sum_numba,
    "NumPy": sum_numpy,
    "NumPy with Numba": sum_numpy_numba,
    "PyTorch": sum_torch,
    "Python Iterative": sum_iterative,
    "Python Built-in": sum_builtin,
}

In [3]:
def benchmark(n=1_000_000, runs=10):
    print("Bencharking with array size:", n)
    a = np.random.rand(n).astype(np.float32)
    results = {}
    for name, func in sum_functions.items():
        func(a)  # warmup
        duration = timeit.timeit(lambda: func(a), number=runs)
        results[name] = duration / runs

    for name, duration in sorted(results.items(), key=lambda item: item[1]):
        print(f"{name}: {duration:.6f} seconds per run")
    print()

In [4]:
benchmark(1_000_000, runs=10)
benchmark(10_000_000, runs=10)
benchmark(10_000, runs=10)
benchmark(100, runs=10)

Bencharking with array size: 1000000
PyTorch: 0.000049 seconds per run
NumPy: 0.000178 seconds per run
NumPy with Numba: 0.000783 seconds per run
Numba Iterative: 0.000784 seconds per run
Python Built-in: 0.077874 seconds per run
Python Iterative: 0.109363 seconds per run

Bencharking with array size: 10000000
PyTorch: 0.001257 seconds per run
NumPy: 0.002404 seconds per run
Numba Iterative: 0.007988 seconds per run
NumPy with Numba: 0.008021 seconds per run
Python Built-in: 0.826637 seconds per run
Python Iterative: 1.135897 seconds per run

Bencharking with array size: 10000
NumPy: 0.000007 seconds per run
NumPy with Numba: 0.000008 seconds per run
Numba Iterative: 0.000008 seconds per run
PyTorch: 0.000011 seconds per run
Python Built-in: 0.001220 seconds per run
Python Iterative: 0.001233 seconds per run

Bencharking with array size: 100
NumPy with Numba: 0.000001 seconds per run
Numba Iterative: 0.000001 seconds per run
NumPy: 0.000005 seconds per run
PyTorch: 0.000008 seconds per

### 1.2 Sum of rows vs columns


#### PyTorch

Usually summing over rows (dim=1) is faster than summing over columns (dim=0) due to memory layout (row-major order).


In [5]:
def benchmark_torch(rows=1000, cols=1000, runs=10):
    device = torch.device("cpu")
    tensor = torch.randn(rows, cols, device=device)
    print(f"Benchmarking sum over array of shape: {rows}x{cols}")
    print(f"Device: {device}")

    t_rows = timeit.timeit(lambda: torch.sum(tensor, dim=1), number=runs)
    print(f"Sum Rows: {t_rows / runs:.6f} seconds per run")

    t_cols = timeit.timeit(lambda: torch.sum(tensor, dim=0), number=runs)
    print(f"Sum Columns: {t_cols / runs:.6f} seconds per run")

    print(f"Rows faster by: {t_cols / t_rows:.2f}x\n")


runs = 50
benchmark_torch(25_000, 25_000, runs=runs)
benchmark_torch(10_000, 10_000, runs=runs)
benchmark_torch(1_000, 1_000, runs=runs)
benchmark_torch(100, 100, runs=runs)
benchmark_torch(100_000, 100, runs=runs)
benchmark_torch(100, 100_000, runs=runs)

Benchmarking sum over array of shape: 25000x25000
Device: cpu
Sum Rows: 0.069855 seconds per run
Sum Columns: 0.249759 seconds per run
Rows faster by: 3.58x

Benchmarking sum over array of shape: 10000x10000
Device: cpu
Sum Rows: 0.010689 seconds per run
Sum Columns: 0.019094 seconds per run
Rows faster by: 1.79x

Benchmarking sum over array of shape: 1000x1000
Device: cpu
Sum Rows: 0.000029 seconds per run
Sum Columns: 0.000033 seconds per run
Rows faster by: 1.13x

Benchmarking sum over array of shape: 100x100
Device: cpu
Sum Rows: 0.000005 seconds per run
Sum Columns: 0.000005 seconds per run
Rows faster by: 0.92x

Benchmarking sum over array of shape: 100000x100
Device: cpu
Sum Rows: 0.001285 seconds per run
Sum Columns: 0.002803 seconds per run
Rows faster by: 2.18x

Benchmarking sum over array of shape: 100x100000
Device: cpu
Sum Rows: 0.001043 seconds per run
Sum Columns: 0.001663 seconds per run
Rows faster by: 1.60x



#### Numpy

Interestingly in Numpy it doesn't work that way.


In [6]:
def benchmark_np(rows=1000, cols=1000, runs=10):
    a = np.random.rand(rows, cols).astype(np.float32)
    print(f"Benchmarking sum over array of shape ({rows}, {cols})")

    t_rows = timeit.timeit(lambda: np.sum(a, axis=1), number=runs)
    print(f"Sum Rows: {t_rows / runs:.6f} seconds per run")

    t_cols = timeit.timeit(lambda: np.sum(a, axis=0), number=runs)
    print(f"Sum Columns: {t_cols / runs:.6f} seconds per run")

    print(f"Rows faster by: {t_cols / t_rows:.2f}x\n")


runs = 50
benchmark_np(25_000, 25_000, runs=runs)
benchmark_np(10_000, 10_000, runs=runs)
benchmark_np(1_000, 1_000, runs=runs)
benchmark_np(100, 100, runs=runs)
benchmark_np(100_000, 100, runs=runs)
benchmark_np(100, 100_000, runs=runs)

Benchmarking sum over array of shape (25000, 25000)
Sum Rows: 0.160197 seconds per run
Sum Columns: 0.163368 seconds per run
Rows faster by: 1.02x

Benchmarking sum over array of shape (10000, 10000)
Sum Rows: 0.027006 seconds per run
Sum Columns: 0.026842 seconds per run
Rows faster by: 0.99x

Benchmarking sum over array of shape (1000, 1000)
Sum Rows: 0.000193 seconds per run
Sum Columns: 0.000142 seconds per run
Rows faster by: 0.73x

Benchmarking sum over array of shape (100, 100)
Sum Rows: 0.000007 seconds per run
Sum Columns: 0.000007 seconds per run
Rows faster by: 1.01x

Benchmarking sum over array of shape (100000, 100)
Sum Rows: 0.003870 seconds per run
Sum Columns: 0.004025 seconds per run
Rows faster by: 1.04x

Benchmarking sum over array of shape (100, 100000)
Sum Rows: 0.002535 seconds per run
Sum Columns: 0.002371 seconds per run
Rows faster by: 0.94x



# 2. Kernel Fusion

Kernel is a function that runs on the GPU.

Often the kernel is memory bound, meaning that the performance is limited by memory bandwidth rather than compute. That means that the kernel spends most of its time waiting for data to be loaded from memory, rather than performing computations. By fusing multiple operations into a single kernel, we can reduce the number of memory accesses, reduce overhead, and improve performance.

We use GELU activation function as an example, which is defined as:
$$\text{GELU}(x) = x \cdot \frac{1}{2} \left(1 + \text{erf}\left(\frac{x}{\sqrt{2}}\right)\right)$$

where $\text{erf}$ is the [error function](https://en.wikipedia.org/wiki/Error_function). This function is commonly used in deep learning models, and it can be implemented in a straightforward way using PyTorch. However, we can also fuse the operations into a single kernel using `torch.compile` to achieve better performance.


In [ ]:
@torch.compile
def gelu_fuse(x):
    return x * 0.5 * (1.0 + torch.erf(x / 1.41421))


def gelu_slow(x):
    return x * 0.5 * (1.0 + torch.erf(x / 1.41421))


x = torch.randn(10_000_000)
# Warmup, torch.compile takes longer in the first iteration, as it must compile the model,
# but in subsequent iterations, we see significant speedups compared to eager.

gelu_fuse(x)
gelu_slow(x)

runs = 10
t_gelu = timeit.timeit(lambda: gelu_fuse(x), number=runs)
t_gelu_slow = timeit.timeit(lambda: gelu_slow(x), number=runs)
print(f"GELU (compiled): {t_gelu / runs:.6f} seconds per run")
print(f"GELU (slow): {t_gelu_slow / runs:.6f} seconds per run")
print(f"Compiled GELU faster by: {t_gelu_slow / t_gelu:.2f}x\n")

GELU (compiled): 0.016426 seconds per run
GELU (slow): 0.055672 seconds per run
Compiled GELU faster by: 3.39x

